<a href="https://colab.research.google.com/github/bmx-ai/phi-playground/blob/main/notebooks/evaluation/evaluation_GSM8k_transformers_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

GSM-8k colab
Author [@fabmilo](https://twitter.com/fabmilo)

In [1]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness --depth 1
%cd lm-evaluation-harness
%pip install -e .

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 2631, done.
remote: Counting objects: 100% (2631/2631), done.
remote: Compressing objects: 100% (1769/1769), done.
remote: Total 2631 (delta 1036), reused 2046 (delta 844), pack-reused 0
Receiving objects: 100% (2631/2631), 1.04 MiB | 9.36 MiB/s, done.
Resolving deltas: 100% (1036/1036), done.
/content/lm-evaluation-harness
Obtaining file:///content/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [5]:
%pip install transformers~=4.37.0 hf_transfer -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.8 MB/s eta 0:00:00


In [3]:
!nvidia-smi

Sun Jan 28 21:07:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
#@title download model
import os

workspace_dir = '/content'
model_path = os.path.join(workspace_dir, '.cache', 'models', 'microsoft', 'phi-2')
if not os.path.exists(model_path):
    !HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download microsoft/phi-2 --local-dir {model_path} --local-dir-use-symlinks False

In [17]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)
del model
del tokenizer

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [1]:
%env NUMEXPR_MAX_THREADS 12

env: NUMEXPR_MAX_THREADS=12


In [5]:
%%time

!lm_eval --model hf      \
    --device cuda:0      \
    --model_args pretrained={model_path} \
    --batch_size 16      \
    --tasks gsm8k        \
    --num_fewshot 8      \
    --output_path lm-eval-outputs/microsoft-phi-2-vanilla \
    --verbosity DEBUG

2024-01-28:21:36:52,110 INFO     [config.py:58] PyTorch version 2.1.0+cu121 available.
2024-01-28:21:36:52,111 INFO     [config.py:95] TensorFlow version 2.15.0 available.
2024-01-28:21:36:52,112 INFO     [config.py:108] JAX version 0.4.23 available.
2024-01-28 21:36:52.624060: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 21:36:52.624111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 21:36:52.625893: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 21:36:53.760529: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=microsoft/phi-1_5 \
    --tasks gsm8k \
    --device cuda:0 \
    --batch_size 16

2024-01-27:00:10:14,629 INFO     [utils.py:148] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-01-27:00:10:14,629 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2024-01-27:00:10:14,892 INFO     [config.py:58] PyTorch version 2.1.0+cu121 available.
2024-01-27:00:10:14,892 INFO     [config.py:95] TensorFlow version 2.15.0 available.
2024-01-27:00:10:14,893 INFO     [config.py:108] JAX version 0.4.23 available.
2024-01-27 00:10:15.383830: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 00:10:15.383920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 00:10:15.385733: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to